In [1]:
import pandas as pd
from exosyspop.populations import TRILEGAL_BGBinaryPopulation
from exosyspop.populations import KeplerBinaryPopulation, PoissonPlanetPopulation

targets = pd.read_hdf('targets.h5')
bgstars = pd.read_hdf('bgstars.h5')

import logging
rootLogger = logging.getLogger()

In [2]:
ebpop = KeplerBinaryPopulation(targets)
bgpop = TRILEGAL_BGBinaryPopulation(targets, bgstars)
plpop = PoissonPlanetPopulation(targets)

In [3]:
class PopulationMixture(object):
    def __init__(self, poplist):
        self.poplist = poplist
        
    @property
    def param_names(self):
        p = []
        for pop in self.poplist:
            p += pop.param_names
        return list(set(p))
    
    def set_params(self, **kwargs):
        for pop in self.poplist:
            pop.set_params(**kwargs)
                        
    def train_trap(self, **kwargs):
        return [p._train_trap(**kwargs) for p in self.poplist]
                
    def observe(self, **kwargs):
        obs = []
        for pop in self.poplist:
            o = pop.observe(new=True, **kwargs)
            o.loc[:, 'population'] = pop.name
            obs.append(o)
            
        return pd.concat(obs)

pops = PopulationMixture([ebpop, bgpop, plpop])

In [4]:
plines = pops.train_trap(N=3000)

EB: dmag regressor trained, R2=0.999571750853
EB: qR regressor trained, R2=0.999498173627
EB: Depth trained: R2=0.998275903724
EB: Duration trained: R2=0.997144917635
EB: Slope trained: R2=0.989480978571
BGEB: dmag regressor trained, R2=0.971850559937
BGEB: qR regressor trained, R2=0.971543462003
BGEB: Depth trained: R2=0.99233604839
BGEB: Duration trained: R2=0.996768785104
BGEB: Slope trained: R2=0.990805214752
Planet: Depth trained: R2=0.998505958327
Planet: Duration trained: R2=0.99823480688
Planet: Slope trained: R2=0.993634913352


In [5]:
obs = pops.observe()
len(obs)

4536

In [6]:
%prun pops.observe(regr_trap=True)

In [7]:
obs.groupby('population').count()

,T14_pri,T14_sec,T23_pri,T23_sec,a,aR,b_pri,b_sec,b_target,d_pri,...,inc,k,n_pri,n_sec,occ,period,phase_sec,target_mag,tra,w
population,,,,,,,,,,,,,,,,,,,,,
BGEB,382,382,382,382,382,382,382,382,382,382,...,382,382,382,382,382,382,382,382,382,382
EB,543,543,543,543,543,543,543,543,543,543,...,543,543,543,543,543,543,543,0,543,543
Planet,3611,3611,3611,3611,3611,3611,3611,3611,3611,3611,...,3611,3611,3611,3611,3611,3611,3611,0,3611,3611
